<a href="https://colab.research.google.com/github/emisoft-designs/Data-Science-Practice-Projects/blob/main/Key%20Informant%20Interview%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gspread
from google.colab import auth,files
auth.authenticate_user()
from oauth2client.client import GoogleCredentials

In [ ]:
# setup
gc = gspread.authorize(GoogleCredentials.get_application_default())

# read data and put it in a dataframe
gsheets1 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1yhTiQmv1Z1lkgXAx-DPqPD1eEC3hRk9AeO3FnABajNk/edit?usp=sharing')
gsheets2 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1AttBD8xVNDszTiCabgHBxMvC72w2pUle4HvWb3FiwO4/edit?usp=sharing')
gsheets3 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Mfo1mSwfCsNp-w2QzG4t68Kclb_hct8ToJNjXIplNd8/edit?usp=sharing')
gsheets4 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1M7I707BKA1WTRVTOrJ_QWypL4qURpije3f4fK0xGta4/edit?usp=sharing')
gsheets5 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1kLNbSpiu4FJdQpBy8Y2lZhk9qrqsJKv0VvV5kJRWy7E/edit?usp=sharing')
gsheets6 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1vnTH6xFjhKruxQfxoApXgAkakhziwyBbL9Eh0WnCcJ4/edit?usp=sharing')
gsheets7 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1pUeSuHJlHL5q86nyEUb0tzMqMkubagTxR9ZF_YeF8rE/edit?usp=sharing')

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sheet1 = pd.DataFrame(gsheets1.worksheet('Form responses 1').get_all_values())
sheet2 = pd.DataFrame(gsheets2.worksheet('Form responses 1').get_all_values())
sheet3 = pd.DataFrame(gsheets3.worksheet('Form responses 1').get_all_values())
sheet4 = pd.DataFrame(gsheets4.worksheet('Form responses 1').get_all_values())
sheet5 = pd.DataFrame(gsheets5.worksheet('Form responses 1').get_all_values())
sheet6 = pd.DataFrame(gsheets6.worksheet('Form responses 1').get_all_values())
sheet7 = pd.DataFrame(gsheets7.worksheet('Form responses 1').get_all_values())

In [ ]:
# reset co

In [ ]:
sample_df = pd.concat([sheet1,sheet2.iloc[1:],sheet3.iloc[1:],sheet4.iloc[1:],sheet5.iloc[1:],sheet6.iloc[1:],sheet7.iloc[1:]],ignore_index=True,axis=0)
sample_df.columns = sample_df.iloc[0] # reset column
sample_df = sample_df.iloc[1:]
sample_df.head(2)

#**FUNCTIONS/DEFINITIONS**

In [ ]:
plt.rc('font', size=12) #controls default text size
plt.rc('axes', titlesize=16) #fontsize of the title
plt.rc('axes', labelsize=14) #fontsize of the x and y labels
plt.rc('xtick', labelsize=12) #fontsize of the x tick labels
plt.rc('ytick', labelsize=12) #fontsize of the y tick labels
plt.rc('legend', fontsize=12) #fontsize of the legend

In [ ]:
def _filter(dic,col): 
  correction_df = pd.DataFrame(dic) # create dataframe from dictionary for mapping
  # loop through column an effect correction
  for w, c in correction_df.itertuples(index=False,name=None):  
    col = col.str.replace(r'\b{0}\b'.format(w),c)
  return col

In [ ]:
def create_df(index,series):
  df = ','.join(x for x in series if type(x) == str)
  df = df.split(',')
  df = [x.replace(';',',') for x in df]
  df = [x.strip() for x in df]
  df_s = pd.Series(df) 
  dfp  = pd.DataFrame(df_s.value_counts()) 
  dfp.columns = ['Responses'] 
  return dfp

In [ ]:
def getCol(df,col):
  df.reset_index(inplace=True)  # reset index
  df.columns = [col,'Responses'] # rename column
  return df # display table

In [ ]:
# convert age to categories of range of 5 years remove unwanted character
def getCategory(df,col,min,max,ran):
  bin = np.arange(min, max, ran)
  labels = ["{0} - {1}".format(i, i + ran-1) for i in bin]
  bin = pd.IntervalIndex.from_tuples([tuple(map(float, x.split('-'))) for x in labels])
  #if col.upper() == 'AGE':
    #bin = pd.IntervalIndex.from_tuples([(0, 5), (6, 10), (11, 15), (16, 20),(21, 25),(26, 30),(31, 35), (36, 40), (41, 45), (46, 50), (51, 55), (56, 60),(61, 65),(66, 70)])
  group = df.groupby(pd.cut(df[col], bin)).sum()
  group = group.drop(col,axis=1)
  group.index = group.index.astype(str)
  if col.upper() == 'AGE':
    group.index = [x.replace(']',')') for x in group.index]
    group.index = [x.replace(', ',' - ') for x in group.index]
    group.index = [x.replace('.0','') for x in group.index]
  else:
    group.index = [x.replace(']','k)') for x in group.index]
    group.index = [x.replace(', ','k - ') for x in group.index]
    group.index = [x.replace('.0','') for x in group.index]
  return group

In [ ]:
def sort_dataframe(df):
  df.columns=['Responses']
  df.index = df.index.astype(int)
  df.sort_index(inplace=True)
  return df

In [ ]:
def restruct(word):
  w = word.strip('# | ,')
  
  if len(word.split('-'))>1: w = word.split('-')[1]

  if len(word.split('.'))>1: 
    if word.split('.')[1] == '00':w = word.split('.')[0]

  if len(word.split(' '))>1: 
    if word.split(' ')[0].isnumeric():
      w = word.split(' ')[0]

  a = ''
  b = ''
  result = ''

  for x in w: 
    if x.isdigit():
      a+=x
    elif x.isalpha() or x.isspace():
      b+=x
  
  b = b.strip()
  a = a.strip()

  if len(b.split(' '))>1:
    br = b.split(' ')
    for v in br:
      if v == 'MILLION' or v == 'NMILLION':
        b = 'MILLION'
        break
      elif v == 'HUNDRED':
        break

  if b == 'MILLION' or b=='NMILLION':
    if len(a)>=2 :a+='00000'
    elif len(a)==1: a+='000000'
  elif b == 'K':
    a+='000'
  
  result = a
  if a == '':
    result = b

  return result

In [ ]:
def Create(data, name='', label=[], level=1, cat=False, min=0, max=1000, range=5, sort=False):
  if level > 2: return 'FAILED'
  
  data[name] = data[name].replace('',np.nan)

  if data[name].dtype == object:
    data.columns = data.columns.str.upper()
    data.columns = data.columns.str.strip()
  
    data['STATE_STUDIED'] = data['STATE_STUDIED'].str.upper()
    data['STATE_STUDIED'] = data['STATE_STUDIED'].str.strip()

    data[name] = data[name].str.upper()
    data[name] = data[name].str.strip() 

  edo_df = data[data['STATE_STUDIED'].str.contains('EDO')]
  kano_df = data[data['STATE_STUDIED'].str.contains('KANO')]
  delta_df = data[data['STATE_STUDIED'].str.contains('DELTA')]
  katsina_df = data[data['STATE_STUDIED'].str.contains('KATSINA')]
  sokoto_df = data[data['STATE_STUDIED'].str.contains('SOKOTO')]
  kaduna_df = data[data['STATE_STUDIED'].str.contains('KADUNA')]
  jigawa_df = data[data['STATE_STUDIED'].str.contains('JIGAWA')]

  
  summary = data

  if level == 0:
      df          = pd.DataFrame(data[name].value_counts())
      df.columns  = ['Responses']
      summary     = df

  elif level == 1:
      df          = pd.DataFrame(data[name].value_counts())
      edo_df      = pd.DataFrame(edo_df[name].value_counts())
      kano_df     = pd.DataFrame(kano_df[name].value_counts())
      delta_df    = pd.DataFrame(delta_df[name].value_counts())
      katsina_df  = pd.DataFrame(katsina_df[name].value_counts())
      sokoto_df   = pd.DataFrame(sokoto_df[name].value_counts())
      kaduna_df   = pd.DataFrame(kaduna_df[name].value_counts())
      jigawa_df   = pd.DataFrame(jigawa_df[name].value_counts())

      if(cat==True):
        if(max<=0):return ''
        df = getCategory(getCol(df,name),name,min,max,range)
        edo_df = getCategory(getCol(edo_df,name),name,min,max,range)
        katsina_df = getCategory(getCol(katsina_df,name),name,min,max,range)
        kaduna_df = getCategory(getCol(kaduna_df,name),name,min,max,range)
        sokoto_df = getCategory(getCol(sokoto_df,name),name,min,max,range)
        delta_df = getCategory(getCol(delta_df,name),name,min,max,range)
        kano_df = getCategory(getCol(kano_df,name),name,min,max,range)
        jigawa_df = getCategory(getCol(jigawa_df,name),name,min,max,range)
      else:
        df.columns          = ['Responses']
        edo_df.columns      = ['Responses']
        delta_df.columns    = ['Responses']
        kano_df.columns     = ['Responses']
        katsina_df.columns  = ['Responses']
        kaduna_df.columns   = ['Responses']
        sokoto_df.columns   = ['Responses']
        jigawa_df.columns   = ['Responses']

      if(sort==True):
        df = sort_dataframe(df)
        edo_df= sort_dataframe(edo_df)
        katsina_df = sort_dataframe(katsina_df)
        kaduna_df = sort_dataframe(kaduna_df)
        sokoto_val = sort_dataframe(sokoto_df)
        delta_val= sort_dataframe(delta_df)
        kano_val = sort_dataframe(kano_df)
        jigawa_val= sort_dataframe(jigawa_df)

  elif level == 2:
      df = create_df(label, data[name])
      edo_df = create_df(label, edo_df[name])
      delta_df = create_df(label, delta_df[name])
      kano_df = create_df(label, kano_df[name])
      katsina_df = create_df(label, katsina_df[name])
      kaduna_df = create_df(label, kaduna_df[name])
      sokoto_df = create_df(label, sokoto_df[name])
      jigawa_df = create_df(label, jigawa_df[name])
  
  if(level>0):
    summary = pd.concat([edo_df,delta_df,kano_df,katsina_df,kaduna_df,sokoto_df,jigawa_df, df], axis=1)
    summary.columns = ['Edo','Delta','Kano','Katsina','Kaduna','Sokoto','Jigawa','Total']
    
  if(sort==False):
    summary['sort'] = [format(str(x)) for x in  summary.index]
    summary =   summary.sort_values('sort').drop('sort', axis=1)

  check = summary.index.isin(['nan'])
  if(check[0]==True):
    summary = summary.drop('nan', axis=0)

  summary.to_csv('/content/drive/MyDrive/Colab Notebooks/KII Analysis/{}_Summary.csv'.format(name))

  return summary

#**DATA WRANGLING**

In [ ]:
sample_df.columns = sample_df.columns.str.upper()
sample_df.columns

###**STATE STUDIED**

In [ ]:
sample_df.STATE_STUDIED.unique()

In [ ]:
sample_df.STATE_STUDIED = sample_df.STATE_STUDIED.str.upper()   # ensure they are all in upper case
sample_df.STATE_STUDIED = [x.replace('STATE','').strip() for x in sample_df.STATE_STUDIED] #strip out "STATE" from all the text and strip out white spaces around
sample_df.STATE_STUDIED = sample_df.STATE_STUDIED.replace('',np.nan) # convert empty spaces to nan values
sample_df.STATE_STUDIED.value_counts()

In [ ]:
state_df = Create(data=sample_df, name='STATE_STUDIED', level=0)
state_df

###**RESPONDENTS' AGE**

In [ ]:
sample_df.AGE.unique()
sample_df.AGE = sample_df.AGE.str.upper()
sample_df.AGE = sample_df.AGE.str.strip()
sample_df.AGE = sample_df.AGE.replace('',np.nan)
sample_df.AGE = pd.to_numeric(sample_df.AGE)
sample_df.AGE.value_counts()

In [ ]:
age_df = Create(sample_df,'AGE',cat=True,max=int(sample_df.AGE.max()))
age_df

###**RESPONDENTS' SEX**

In [ ]:
sample_df.SEX.unique()
sample_df.SEX = sample_df.SEX.str.upper()
sample_df.SEX = sample_df.SEX.str.strip()
sample_df.SEX = sample_df.SEX.replace('',np.nan)
sample_df.SEX.value_counts()

In [ ]:
sex_df = Create(sample_df,'SEX')
sex_df

###**RESPONDENTS' MARITAL STATUS**

In [ ]:
sample_df.MARITAL_STATUS.unique()

In [ ]:
sample_df.MARITAL_STATUS = sample_df.MARITAL_STATUS.str.upper()
sample_df.MARITAL_STATUS = sample_df.MARITAL_STATUS.str.strip()
sample_df.MARITAL_STATUS = sample_df.MARITAL_STATUS.replace('',np.nan)
sample_df.MARITAL_STATUS.value_counts()

In [ ]:
marital_df = Create(sample_df,'MARITAL_STATUS')
marital_df

###**RESPONDENTS' DEPENDANTS**

In [ ]:
sample_df.DEPENDANTS.unique()

array(['4', '3', '5', '1', '2', '6', '0', '', '7', '11', '10', '13', '8',
       '24', '31'], dtype=object)

In [ ]:
sample_df.DEPENDANTS = sample_df.DEPENDANTS.str.strip()
sample_df.DEPENDANTS = sample_df.DEPENDANTS.replace('',np.nan) 
sample_df.DEPENDANTS.value_counts()

4     16
3     12
2     12
1     11
5      9
0      6
6      5
7      3
8      3
11     2
10     2
13     2
24     1
31     1
Name: DEPENDANTS, dtype: int64

In [ ]:
dep_df = Create(sample_df,'DEPENDANTS', level=1, sort=True)
dep_df

,Edo,Delta,Kano,Katsina,Kaduna,Sokoto,Jigawa,Total
0,5.0,NaN,NaN,NaN,NaN,1.0,NaN,6
1,4.0,1.0,NaN,NaN,5.0,NaN,1.0,11
2,5.0,3.0,1.0,NaN,1.0,1.0,1.0,12
3,2.0,5.0,1.0,2.0,1.0,1.0,NaN,12
4,5.0,2.0,3.0,3.0,1.0,1.0,1.0,16
5,2.0,3.0,2.0,NaN,NaN,NaN,2.0,9
6,NaN,1.0,NaN,1.0,2.0,NaN,1.0,5
7,NaN,NaN,1.0,NaN,1.0,NaN,1.0,3
8,NaN,NaN,2.0,1.0,NaN,NaN,NaN,3
10,NaN,NaN,2.0,NaN,NaN,NaN,NaN,2


###**RESPONDENTS' EDUCATION**

In [ ]:
sample_df.EDUCATION.unique()

array(["HND or Bachelor's degree", 'Secondary School', "Master's degree",
       'Primary School', '', 'Diploma', 'None', 'Informal Schooling'],
      dtype=object)

In [ ]:
sample_df.EDUCATION = sample_df.EDUCATION.str.strip()
sample_df.EDUCATION = sample_df.EDUCATION.replace('',np.nan) 
sample_df.EDUCATION.value_counts()

Secondary School            42
Primary School              13
Diploma                      9
HND or Bachelor's degree     8
Informal Schooling           6
None                         2
Master's degree              1
Name: EDUCATION, dtype: int64

In [ ]:
edu_df = Create(sample_df,'EDUCATION')
edu_df

,Edo,Delta,Kano,Katsina,Kaduna,Sokoto,Jigawa,Total
DIPLOMA,1.0,NaN,3.0,NaN,1.0,3.0,1.0,9
HND OR BACHELOR'S DEGREE,1.0,5.0,NaN,1.0,1.0,NaN,NaN,8
INFORMAL SCHOOLING,NaN,NaN,NaN,4.0,1.0,1.0,NaN,6
MASTER'S DEGREE,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1
NONE,NaN,NaN,1.0,NaN,NaN,NaN,1.0,2
PRIMARY SCHOOL,2.0,2.0,3.0,2.0,1.0,1.0,2.0,13
SECONDARY SCHOOL,17.0,6.0,6.0,4.0,5.0,NaN,4.0,42


###**RESPONDENTS' RESIDENT STATE**

In [ ]:
sample_df.LOCATION.unique()

array(['DELTA', '', 'EDO ', 'EDO', 'EDO STATE', 'BENIN', 'IRRUA', 'AUCHI',
       'UROMI', 'BENIN CITY', 'JIGAWA', 'KANO', 'KADUNA', 'KATSINA',
       'KATSINA ', 'SOKOTO', 'ZAMFARA'], dtype=object)

In [ ]:
sample_df.LOCATION = sample_df.LOCATION.replace(['DELTA', '', 'EDO ', 'EDO', 'EDO STATE', 'BENIN', 'IRRUA', 'AUCHI',
       'UROMI', 'BENIN CITY', 'JIGAWA', 'KANO', 'KADUNA', 'KATSINA',
       'KATSINA ', 'SOKOTO', 'ZAMFARA'],['DELTA', '', 'EDO', 'EDO', 'EDO STATE', 'EDO', 'EDO', 'EDO',
       'EDO', 'EDO', 'JIGAWA', 'KANO', 'KADUNA', 'KATSINA',
       'KATSINA', 'SOKOTO', 'ZAMFARA'])

In [ ]:
sample_df.LOCATION = sample_df.LOCATION.str.upper()   # ensure they are all in upper case
sample_df.LOCATION = [x.replace('STATE','').strip() for x in sample_df.LOCATION] #strip out "STATE" from all the text and strip out white spaces around
sample_df.LOCATION = sample_df.LOCATION.replace('',np.nan) # convert empty spaces to nan values
sample_df.LOCATION.value_counts()

EDO        24
DELTA      14
KANO       13
KADUNA     13
KATSINA    11
JIGAWA      7
SOKOTO      6
ZAMFARA     1
Name: LOCATION, dtype: int64

In [ ]:
loc_df = Create(sample_df,'LOCATION')
loc_df

,Edo,Delta,Kano,Katsina,Kaduna,Sokoto,Jigawa,Total
DELTA,NaN,14.0,NaN,NaN,NaN,NaN,NaN,14
EDO,24.0,NaN,NaN,NaN,NaN,NaN,NaN,24
JIGAWA,NaN,NaN,NaN,NaN,NaN,NaN,7.0,7
KADUNA,NaN,NaN,NaN,NaN,13.0,NaN,NaN,13
KANO,NaN,NaN,12.0,NaN,NaN,NaN,1.0,13
KATSINA,NaN,NaN,NaN,11.0,NaN,NaN,NaN,11
SOKOTO,NaN,NaN,NaN,NaN,NaN,6.0,NaN,6
ZAMFARA,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1


###**RESPONDENTS' ORIGIN STATE**

In [ ]:
sample_df.ORIGIN_STATE.unique()

array(['DELTA', 'EDO', 'IMO', 'ENUGU', 'JIGAWA', 'KANO', 'KADUNA',
       'NIGER', 'OYO', '', 'KATSINA', 'ZAMFARA', 'SOKOTO'], dtype=object)

In [ ]:
sample_df.ORIGIN_STATE = sample_df.ORIGIN_STATE.str.strip()
sample_df.ORIGIN_STATE = sample_df.ORIGIN_STATE.replace('',np.nan) 
sample_df.ORIGIN_STATE.value_counts()

EDO        23
DELTA      14
KANO       13
JIGAWA      9
KADUNA      9
KATSINA     7
SOKOTO      7
ENUGU       3
IMO         2
NIGER       1
OYO         1
ZAMFARA     1
Name: ORIGIN_STATE, dtype: int64

In [ ]:
origin_df = Create(sample_df,'ORIGIN_STATE')
origin_df

,Edo,Delta,Kano,Katsina,Kaduna,Sokoto,Jigawa,Total
DELTA,1.0,13.0,NaN,NaN,NaN,NaN,NaN,14
EDO,21.0,1.0,NaN,NaN,1.0,NaN,NaN,23
ENUGU,1.0,NaN,NaN,NaN,2.0,NaN,NaN,3
IMO,1.0,1.0,NaN,NaN,NaN,NaN,NaN,2
JIGAWA,NaN,NaN,1.0,1.0,NaN,NaN,7.0,9
KADUNA,NaN,NaN,NaN,1.0,8.0,NaN,NaN,9
KANO,NaN,NaN,11.0,1.0,NaN,NaN,1.0,13
KATSINA,NaN,NaN,NaN,6.0,NaN,1.0,NaN,7
NIGER,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1
OYO,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1


###**RESPONDENTS' COUNTRY OF ORIGIN**

In [ ]:
sample_df.COUNTRY.unique()

array(['NIGERIA', '', ' NIGERIA'], dtype=object)

In [ ]:
sample_df.COUNTRY = sample_df.COUNTRY.str.strip()
sample_df.COUNTRY = sample_df.COUNTRY.replace('',np.nan) 
sample_df.COUNTRY.value_counts()

NIGERIA    90
Name: COUNTRY, dtype: int64

In [ ]:
country_df = Create(sample_df,'COUNTRY')
country_df

,Edo,Delta,Kano,Katsina,Kaduna,Sokoto,Jigawa,Total
NIGERIA,24,15,12,11,13,7,8,90


###**RESPONDENTS' JOB STATUS**

In [ ]:
sample_df.JOB_STATUS.unique()

array(['Self-employed', 'Not Working', 'Working', 'Seeking for Job'],
      dtype=object)

In [ ]:
sample_df.JOB_STATUS = sample_df.JOB_STATUS.str.strip()
sample_df.JOB_STATUS = sample_df.JOB_STATUS.replace('',np.nan) 
sample_df.JOB_STATUS.value_counts()

Self-employed      61
Seeking for Job    15
Not Working         8
Working             7
Name: JOB_STATUS, dtype: int64

In [ ]:
job_df = Create(sample_df,'JOB_STATUS')
job_df

,Edo,Delta,Kano,Katsina,Kaduna,Sokoto,Jigawa,Total
NOT WORKING,1,5.0,NaN,NaN,NaN,1.0,1.0,8
SEEKING FOR JOB,4,NaN,NaN,2.0,7.0,2.0,NaN,15
SELF-EMPLOYED,16,9.0,13.0,8.0,4.0,4.0,7.0,61
WORKING,3,1.0,NaN,1.0,2.0,NaN,NaN,7


###**RESPONDENTS' OCCUPATION**

In [ ]:
sample_df.OCCUPATION.unique()

array(['BARBER', 'TRADER', 'TAILOR', 'PROVISION', 'BUSINESS', '', 'ACTOR',
       'CIVIL SERVANT', 'POS BUSINESS', 'FASHION DESIGNER', 'NOTHING ',
       'PHARMACY', 'CARPENTARY/TILING', 'SALES REP',
       'AUTOMOBILE ENGINEERING ', 'STORE KEEPER IN AN ORGANIZATION',
       'SALES OF PROVISION', 'HAIR DRESSER', 'PHOTOGRAPHER/VIDEOGRAPHER',
       'WELDING', 'RUN  A BOUTIQUE', 'CLOTHESE SALES (BOUTIQUE)',
       'FULL HOUSE WIFE', 'CARPENTER', 'BARBAR', 'BRICK LAYER',
       'PASTORING', 'TRADING', 'LOCAL STARTCH SUPPLIER',
       'SELLING WRAPPER', 'MECHANIC', 'TAILORING', 'OKADA', 'SALES HELP',
       'CAPENTER', 'FARMING', 'SELLING PHONE AND ACCESSORIES', 'OIL',
       'DRIVER', 'SEWING', 'MARKETER', 'RESTURANT ATTENDANT', 'MARCHANT',
       'PETTY TRADING', 'DEALER OF USED CLOTHS', 'SELLING FOOD'],
      dtype=object)

In [ ]:
sample_df.OCCUPATION = sample_df.OCCUPATION.str.strip()
sample_df.OCCUPATION = sample_df.OCCUPATION.replace('',np.nan) 
sample_df.OCCUPATION.value_counts()

BUSINESS                           23
TRADER                              5
SALES REP                           2
PETTY TRADING                       2
CIVIL SERVANT                       2
DRIVER                              2
HAIR DRESSER                        2
CAPENTER                            1
LOCAL STARTCH SUPPLIER              1
SELLING WRAPPER                     1
MECHANIC                            1
TAILORING                           1
OKADA                               1
SALES HELP                          1
BARBER                              1
FARMING                             1
SELLING PHONE AND ACCESSORIES       1
PASTORING                           1
OIL                                 1
SEWING                              1
MARKETER                            1
RESTURANT ATTENDANT                 1
MARCHANT                            1
DEALER OF USED CLOTHS               1
TRADING                             1
CARPENTER                           1
BRICK LAYER 

In [ ]:
sample_df['OCCUPATION_CLASS'] = sample_df.OCCUPATION
sample_df['OCCUPATION_CLASS'].unique()

array(['BARBER', 'TRADER', 'TAILOR', 'PROVISION', 'BUSINESS', nan,
       'ACTOR', 'CIVIL SERVANT', 'POS BUSINESS', 'FASHION DESIGNER',
       'NOTHING', 'PHARMACY', 'CARPENTARY/TILING', 'SALES REP',
       'AUTOMOBILE ENGINEERING', 'STORE KEEPER IN AN ORGANIZATION',
       'SALES OF PROVISION', 'HAIR DRESSER', 'PHOTOGRAPHER/VIDEOGRAPHER',
       'WELDING', 'RUN  A BOUTIQUE', 'CLOTHESE SALES (BOUTIQUE)',
       'FULL HOUSE WIFE', 'CARPENTER', 'BARBAR', 'BRICK LAYER',
       'PASTORING', 'TRADING', 'LOCAL STARTCH SUPPLIER',
       'SELLING WRAPPER', 'MECHANIC', 'TAILORING', 'OKADA', 'SALES HELP',
       'CAPENTER', 'FARMING', 'SELLING PHONE AND ACCESSORIES', 'OIL',
       'DRIVER', 'SEWING', 'MARKETER', 'RESTURANT ATTENDANT', 'MARCHANT',
       'PETTY TRADING', 'DEALER OF USED CLOTHS', 'SELLING FOOD'],
      dtype=object)

In [ ]:
words = ['BARBER', 'TRADER', 'TAILOR', 'PROVISION', 'BUSINESS',
       'ACTOR', 'CIVIL SERVANT', 'POS BUSINESS', 'FASHION DESIGNER',
       'NOTHING', 'PHARMACY', 'CARPENTARY/TILING', 'SALES REP',
       'AUTOMOBILE ENGINEERING', 'STORE KEEPER IN AN ORGANIZATION',
       'SALES OF PROVISION', 'HAIR DRESSER', 'PHOTOGRAPHER/VIDEOGRAPHER',
       'WELDING', 'RUN  A BOUTIQUE', 'CLOTHESE SALES (BOUTIQUE)',
       'FULL HOUSE WIFE', 'CARPENTER', 'BARBAR', 'BRICK LAYER',
       'PASTORING', 'TRADING', 'LOCAL STARTCH SUPPLIER',
       'SELLING WRAPPER', 'MECHANIC', 'TAILORING', 'OKADA', 'SALES HELP',
       'CAPENTER', 'FARMING', 'SELLING PHONE AND ACCESSORIES', 'OIL',
       'DRIVER', 'SEWING', 'MARKETER', 'RESTURANT ATTENDANT', 'MARCHANT',
       'PETTY TRADING', 'DEALER OF USED CLOTHS', 'SELLING FOOD']
grouped = ['CRAFT', 'TRADER', 'CRAFT', 'TRADER', 'TRADER',
       'PROFESSIONAL', 'PROFESSIONAL', 'SERVICE', 'CRAFT',
       'NOTHING', 'PROFESSIONAL', 'CRAFT', 'SALES',
       'ASSOCIATE PROFESSIONAL', 'ASSOCIATE PROFESSIONAL',
       'SALES', 'CRAFT', 'PROFESSIONAL',
       'CRAFT', 'TRADER', 'TRADER',
       'ELEMENTARY', 'CRAFT', 'CRAFT', 'CRAFT',
       'CLERK', 'TRADER', 'SALES',
       'TRADER', 'TECHNICIAN', 'CRAFT', 'SERVICE', 'SALES',
       'CRAFT', 'AGRICULTURAL', 'TRADER', 'TRADER',
       'SERVICE', 'CRAFT', 'SALES', 'SALES', 'TRADER',
       'TRADER', 'TRADER', 'TRADER']
sample_df.OCCUPATION_CLASS = sample_df.OCCUPATION_CLASS.replace(words,grouped)

In [ ]:
sample_df.OCCUPATION_CLASS = sample_df.OCCUPATION_CLASS.str.strip()
sample_df.OCCUPATION_CLASS = sample_df.OCCUPATION_CLASS.replace('',np.nan) 
sample_df.OCCUPATION_CLASS.value_counts()

TRADER                    40
CRAFT                     13
SALES                      7
PROFESSIONAL               5
SERVICE                    4
ASSOCIATE PROFESSIONAL     2
NOTHING                    1
ELEMENTARY                 1
CLERK                      1
TECHNICIAN                 1
AGRICULTURAL               1
Name: OCCUPATION_CLASS, dtype: int64

In [ ]:
occ_df = Create(sample_df,'OCCUPATION_CLASS')
occ_df

,Edo,Delta,Kano,Katsina,Kaduna,Sokoto,Jigawa,Total
AGRICULTURAL,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1
ASSOCIATE PROFESSIONAL,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2
CLERK,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1
CRAFT,7.0,3.0,2.0,NaN,1.0,NaN,NaN,13
ELEMENTARY,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1
NOTHING,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1
PROFESSIONAL,2.0,2.0,NaN,1.0,NaN,NaN,NaN,5
SALES,3.0,NaN,NaN,2.0,2.0,NaN,NaN,7
SERVICE,NaN,1.0,2.0,NaN,1.0,NaN,NaN,4
TECHNICIAN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1


###**RESPONDENTS' INCOME RANGE**

In [ ]:
sample_df.INCOME_RANGE.unique()

array(['N50,000 - N99,999', '', 'below N5,000', 'N5,000 - N49,999',
       'N100,000 - N199,999', 'above N200,000'], dtype=object)

In [ ]:
sample_df.INCOME_RANGE = sample_df.INCOME_RANGE.str.strip()
sample_df.INCOME_RANGE = sample_df.INCOME_RANGE.replace('',np.nan) 
sample_df.INCOME_RANGE.value_counts()

N5,000 - N49,999       43
below N5,000           24
N50,000 - N99,999       5
N100,000 - N199,999     2
above N200,000          2
Name: INCOME_RANGE, dtype: int64

In [ ]:
income_df = Create(sample_df,'INCOME_RANGE')
income_df

,Edo,Delta,Kano,Katsina,Kaduna,Sokoto,Jigawa,Total
"ABOVE N200,000",NaN,NaN,NaN,2.0,NaN,NaN,NaN,2
"BELOW N5,000",3.0,5.0,NaN,1.0,8.0,1.0,6.0,24
"N100,000 - N199,999",NaN,NaN,NaN,2.0,NaN,NaN,NaN,2
"N5,000 - N49,999",14.0,3.0,13.0,6.0,3.0,2.0,2.0,43
"N50,000 - N99,999",4.0,1.0,NaN,NaN,NaN,NaN,NaN,5


In [ ]:
sample_df = sample_df.reindex(columns=['TIMESTAMP', 'CONCEDED', 'STATE_STUDIED', 'LGA', 'PHONE', 'FULL_NAME',
       'FORM_STATUS', 'AGE', 'SEX', 'MARITAL_STATUS', 'DEPENDANTS',
       'EDUCATION', 'LOCATION', 'ORIGIN_STATE', 'COUNTRY', 'JOB_STATUS',
       'OCCUPATION', 'OCCUPATION_CLASS', 'INCOME_RANGE', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7',
       'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17',
       'Q18', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'Q27',
       'Q28', 'Q29', 'Q30', 'Q31'])

In [ ]:
sample_df.columns

Index(['TIMESTAMP', 'CONCEDED', 'STATE_STUDIED', 'LGA', 'PHONE', 'FULL_NAME',
       'FORM_STATUS', 'AGE', 'SEX', 'MARITAL_STATUS', 'DEPENDANTS',
       'EDUCATION', 'LOCATION', 'ORIGIN_STATE', 'COUNTRY', 'JOB_STATUS',
       'OCCUPATION', 'OCCUPATION_CLASS', 'INCOME_RANGE', 'Q1', 'Q2', 'Q3',
       'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14',
       'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24',
       'Q25', 'Q26', 'Q27', 'Q28', 'Q29', 'Q30', 'Q31'],
      dtype='object', name=0)

#**RESPONDENTS' INTERVIEW SECTION**

###**RESPONDENTS' INTERVIEW QUESTION 1**

In [ ]:
sample_df.Q1.unique()

array(['Yes', 'No'], dtype=object)

In [ ]:
sample_df.Q1 = sample_df.Q1.str.strip()
sample_df.Q1 = sample_df.Q1.replace('',np.nan) 
sample_df.Q1.value_counts()

Yes    83
No      8
Name: Q1, dtype: int64

In [ ]:
Q1_df = Create(sample_df,'Q1')
Q1_df

,Edo,Delta,Kano,Katsina,Kaduna,Sokoto,Jigawa,Total
NO,NaN,NaN,2,1,NaN,NaN,5,8
YES,24.0,15.0,11,10,13.0,7.0,3,83


###**RESPONDENTS' INTERVIEW QUESTION 2**




In [ ]:
sample_df.Q2.unique()

array(['Italy', 'Libya', 'Libya, Morocco, EGYPT', 'TRIPOLI', 'MALI',
       'Mali', 'Morocco', 'Libya, Chad', 'Chad', 'EGYPT', 'Algeria', '',
       'Niger', 'Italy, Libya', 'Saudi Arabia', 'Spain', 'Dubai', 'SUDAN',
       'Libya, Mali', 'TOGO', 'Ghana'], dtype=object)

In [ ]:
sample_df.Q2 = sample_df.Q2.replace(['Dubai','TRIPOLI'],['UAE','Libya'])

In [ ]:
sample_df.Q2 = sample_df.Q2.str.strip()
sample_df.Q2 = sample_df.Q2.replace('',np.nan) 
sample_df.Q2.value_counts()

Libya                    47
Algeria                  10
Niger                     5
Chad                      4
Italy                     4
Saudi Arabia              3
SUDAN                     3
Morocco                   2
Mali                      2
Libya, Chad               1
EGYPT                     1
MALI                      1
Libya, Morocco, EGYPT     1
Italy, Libya              1
Spain                     1
UAE                       1
Libya, Mali               1
TOGO                      1
Ghana                     1
Name: Q2, dtype: int64

In [ ]:
Q2_df = Create(sample_df,'Q2',level=2)
Q2_df

,Edo,Delta,Kano,Katsina,Kaduna,Sokoto,Jigawa,Total
ALGERIA,NaN,NaN,4.0,3.0,NaN,1.0,2.0,10
CHAD,NaN,NaN,NaN,NaN,1.0,NaN,4.0,5
EGYPT,NaN,1.0,NaN,NaN,NaN,NaN,1.0,2
GHANA,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1
ITALY,2.0,2.0,NaN,NaN,1.0,NaN,NaN,5
LIBYA,19.0,13.0,9.0,3.0,4.0,1.0,2.0,51
MALI,2.0,NaN,NaN,2.0,NaN,NaN,NaN,4
MOROCCO,1.0,1.0,NaN,NaN,1.0,NaN,NaN,3
NIGER,NaN,NaN,NaN,NaN,2.0,3.0,NaN,5
SAUDI ARABIA,NaN,NaN,NaN,1.0,2.0,NaN,NaN,3


###**RESPONDENTS' INTERVIEW QUESTION 3**

In [ ]:
sample_df.Q3.unique()

array(['No', 'Yes', ''], dtype=object)

In [ ]:
sample_df.Q3 = sample_df.Q3.str.strip()
sample_df.Q3 = sample_df.Q3.replace('',np.nan) 
sample_df.Q3.value_counts()

No     43
Yes    39
Name: Q3, dtype: int64

In [ ]:
Q3_df = Create(sample_df,'Q3')
Q3_df

,Edo,Delta,Kano,Katsina,Kaduna,Sokoto,Jigawa,Total
NO,11,6,12.0,1,5,2,6,43
YES,12,6,NaN,10,4,5,2,39


###**RESPONDENTS' INTERVIEW QUESTION 4**

In [ ]:
sample_df.Q4.unique()

array(['', 'Kano', 'Kano, Sokoto', 'Sokoto', 'KADUNA', 'Katsina',
       'Kano, Katsina', 'Kano, Katsina, KADUNA', 'GHANA', 'Jigawa',
       'Kano, Jigawa', 'BORNO', 'Kebbi'], dtype=object)

In [ ]:
sample_df.Q4 = sample_df.Q4.str.strip()
sample_df.Q4 = sample_df.Q4.replace('GHANA',np.nan) 
sample_df.Q4 = sample_df.Q4.replace('',np.nan) 
sample_df.Q4.value_counts()

Katsina                  27
Kano                     26
Sokoto                   17
Jigawa                    3
Kebbi                     3
Kano, Katsina, KADUNA     2
BORNO                     2
Kano, Sokoto              1
KADUNA                    1
Kano, Katsina             1
Kano, Jigawa              1
Name: Q4, dtype: int64


= pd.DataFrame(df)
df.columns = ['Responses']
df = df.reset_index()
df = df.replace('',np.nan)
df = df.dropna(how='any')
df = df[['STATE_STUDIED','SEX']].value_counts()

In [ ]:
df = sample_df.groupby(['STATE_STUDIED','SEX'])['AGE'].value_counts()
df = pd.DataFrame(df)
df.columns = ['Responses']
df = df.reset_index()
df = df.replace('',np.nan)
df = df.dropna(how='any')
df = df.groupby('STATE_STUDIED')['SEX'].value_counts()
df.to_csv('/content/drive/MyDrive/Colab Notebooks/KII Analysis/{}_Summary.csv'.format('Grouped response'))

In [ ]:
Q4_df = Create(sample_df,'Q4',level=2)
Q4_df

,Edo,Delta,Kano,Katsina,Kaduna,Sokoto,Jigawa,Total
BORNO,NaN,NaN,2.0,NaN,NaN,NaN,NaN,2
JIGAWA,NaN,NaN,NaN,NaN,2.0,NaN,2.0,4
KADUNA,2.0,1.0,NaN,NaN,NaN,NaN,NaN,3
KANO,14.0,11.0,NaN,1.0,5.0,NaN,NaN,31
KATSINA,5.0,NaN,9.0,10.0,4.0,1.0,1.0,30
KEBBI,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3
SOKOTO,7.0,3.0,2.0,NaN,3.0,3.0,NaN,18


###**RESPONDENTS' INTERVIEW QUESTION 5**

In [ ]:
sample_df.Q5.unique()

array(['', 'No', 'Yes'], dtype=object)

In [ ]:
sample_df.Q5 = sample_df.Q5.str.strip()
sample_df.Q5 = sample_df.Q5.replace('',np.nan) 
sample_df.Q5.value_counts()

Yes    67
No     16
Name: Q5, dtype: int64

In [ ]:
Q5_df = Create(sample_df,'Q5')
Q5_df

,Edo,Delta,Kano,Katsina,Kaduna,Sokoto,Jigawa,Total
NO,6,2,2,NaN,NaN,NaN,6,16
YES,18,6,11,11.0,13.0,6.0,2,67


###**RESPONDENTS' INTERVIEW QUESTION 6**

In [ ]:
sample_df.Q6.unique()

array(['FRIEND', 'THROUGH RELATIVES', '', 'SOMEONE FROM BENIN', 'FRIENDS',
       'A CUSTOMER', 'PASTORS CONNECTION', 'FRIENDS ABROAD',
       'BY CONNECTION', 'SOCIAL MEDIA ', 'FRIENDS / BORGER ',
       'FRIENDS ABROAD, BORGER', 'SOCIAL MEDIA', 'THROUGH CONNECTION MAN',
       'SOCIAL MEDIA, BORGER ', 'SOCIAL', 'BORGER', '-',
       'FRIENDS, FAMILY', 'SOCIAL MEDIA, FRIENDS, FAMILY',
       'FRIENDS/FAMILY', 'FRIENDS, FAMILY, SOCIAL MEDIA',
       'FRIENDS, FAMILY ', 'FRIENDS/FAMILY MEMBERS',
       'FRIENDS/SOCIAL MEDIA', 'SOCIAL MEDIA FRIENDS', 'THROUGH OTHERS',
       'I DONT NO', 'SOMEONE WHO LIVES THERE', 'THROUGH HUSBAND',
       'BY AGENT', 'RETURNEES', 'THROUGH AGENT', 'AGENT',
       'FRIENDS AND COLLEAGUES', 'INFORMED BY FRIENDS',
       'ASKED PEOPLE WHO HAVE TRAVELLED BEFORE', 'INFORMED BY PEOPLE',
       'INFORMED BY OTHER PEOPLE', 'BY FRIENDS',
       'A FRIEND WHO HAS TRAVELLED BEFORE', 'MOSTLY AGENTS',
       'THROUGH AGENTS', 'AGENTS AND MARCHANTS', 'AGENTS',
       

In [ ]:
words = ['FRIEND', 'THROUGH RELATIVES', '', 'SOMEONE FROM BENIN', 'FRIENDS',
       'A CUSTOMER', 'PASTORS CONNECTION', 'FRIENDS ABROAD',
       'BY CONNECTION', 'SOCIAL MEDIA ', 'FRIENDS / BORGER ',
       'FRIENDS ABROAD, BORGER', 'SOCIAL MEDIA', 'THROUGH CONNECTION MAN',
       'SOCIAL MEDIA, BORGER ', 'SOCIAL', 'BORGER', '-',
       'FRIENDS, FAMILY', 'SOCIAL MEDIA, FRIENDS, FAMILY',
       'FRIENDS/FAMILY', 'FRIENDS, FAMILY, SOCIAL MEDIA',
       'FRIENDS, FAMILY ', 'FRIENDS/FAMILY MEMBERS',
       'FRIENDS/SOCIAL MEDIA', 'SOCIAL MEDIA FRIENDS', 'THROUGH OTHERS',
       'I DONT NO', 'SOMEONE WHO LIVES THERE', 'THROUGH HUSBAND',
       'BY AGENT', 'RETURNEES', 'THROUGH AGENT', 'AGENT',
       'FRIENDS AND COLLEAGUES', 'INFORMED BY FRIENDS',
       'ASKED PEOPLE WHO HAVE TRAVELLED BEFORE', 'INFORMED BY PEOPLE',
       'INFORMED BY OTHER PEOPLE', 'BY FRIENDS',
       'A FRIEND WHO HAS TRAVELLED BEFORE', 'MOSTLY AGENTS',
       'THROUGH AGENTS', 'AGENTS AND MARCHANTS', 'AGENTS',
       'FRIENDS AND AGENTS', 'AGENTS AND FRIENDS', 'AGENT FRIEND',
       'BY DRIVING', 'BY DRIVERS', 'BY FRIEND']

correction = ['FRIENDS', 'FAMILY', np.nan, 'AGENTS', 'FRIENDS',
       'FRIENDS', 'FRIENDS', 'FRIENDS',
       'AGENTS', 'SOCIAL MEDIA ', 'FRIENDS,BORGER ',
       'FRIENDS, BORGER', 'SOCIAL MEDIA', 'AGENTS',
       'SOCIAL MEDIA, BORGER ', 'SOCIAL MEDIA', 'BORGER', '-',
       'FRIENDS, FAMILY', 'SOCIAL MEDIA, FRIENDS, FAMILY',
       'FRIENDS, FAMILY', 'FRIENDS, FAMILY, SOCIAL MEDIA',
       'FRIENDS, FAMILY ', 'FRIENDS, FAMILY',
       'FRIENDS, SOCIAL MEDIA', 'SOCIAL MEDIA, FRIENDS', 'FRIENDS',
       'NONE', 'FRIENDS', 'FAMILY',
       'AGENTS', 'RETURNEES', 'AGENTS', 'AGENTS',
       'FRIENDS, COLLEAGUES', 'FRIENDS',
       'RETURNEES', 'FRIENDS',
       'FRIENDS', 'FRIENDS',
       'FRIENDS', 'AGENTS',
       'AGENTS', 'AGENTS', 'AGENTS',
       'FRIENDS, AGENTS', 'AGENTS, FRIENDS', 'AGENTS, FRIENDS',
       'DRIVERS', 'DRIVERS', 'FRIENDS']
sample_df.Q6 = sample_df.Q6.replace(words,correction)

In [ ]:
sample_df.Q6 = sample_df.Q6.str.strip()
sample_df.Q6 = sample_df.Q6.replace('',np.nan) 
sample_df.Q6.value_counts()

FRIENDS                          37
AGENTS                           16
FRIENDS, FAMILY                   6
SOCIAL MEDIA                      5
AGENTS, FRIENDS                   3
RETURNEES                         2
FAMILY                            2
DRIVERS                           2
-                                 1
BORGER                            1
SOCIAL MEDIA, BORGER              1
FRIENDS, FAMILY, SOCIAL MEDIA     1
FRIENDS, SOCIAL MEDIA             1
SOCIAL MEDIA, FRIENDS             1
NONE                              1
FRIENDS, BORGER                   1
FRIENDS, COLLEAGUES               1
FRIENDS, AGENTS                   1
FRIENDS,BORGER                    1
SOCIAL MEDIA, FRIENDS, FAMILY     1
Name: Q6, dtype: int64

In [ ]:
Q6_df = Create(sample_df,'Q6',level=2)
Q6_df

,Edo,Delta,Kano,Katsina,Kaduna,Sokoto,Jigawa,Total
-,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1
AGENTS,1.0,2.0,NaN,9.0,7.0,1.0,NaN,20
BORGER,4.0,NaN,NaN,NaN,NaN,NaN,NaN,4
COLLEAGUES,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1
DRIVERS,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2
FAMILY,8.0,1.0,NaN,NaN,NaN,NaN,1.0,10
FRIENDS,15.0,10.0,12.0,4.0,5.0,2.0,6.0,54
NONE,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1
RETURNEES,NaN,NaN,1.0,NaN,1.0,NaN,NaN,2
SOCIAL MEDIA,10.0,NaN,NaN,NaN,NaN,NaN,NaN,10


###**RESPONDENTS' INTERVIEW QUESTION 7**

In [ ]:
sample_df.Q7.unique()

array(['POOR BACKGROUND', 'PRESSURE', 'HARDSHIP', 'BETTER LIFE', '',
       'SEEKING GREENER PASTURE', 'TO MAKE MONEY TO CATER FOR MY FAMILY',
       'TOO MUCH SUFFERING', 'RECESSION', 'PASTORS PROPHESY',
       'TO BETTER MY LIFE', 'FRIENDS ', 'SOCIAL MEDIA ',
       'GREENER PASTURE', 'HIGH RATE OF EURO', 'POVERTY ',
       'ECONOMIC REASONS ', 'FAMILY', 'IT IS CHEAPER',
       'LACK OF DOCUMENTS', 'LOW INCOME TO TRAVEL', 'NO TRAVEL DOCUMENT',
       'INCOMPLETE TRAVEL DOCUMENTS', 'LACK OF MONEY',
       'LACK OF TRAVEL DOCUMENTS', 'POVERTY', 'SUFFERING', 'ITS EASIER',
       'SOCIAL MEDIA', 'LACK OF TRAVEL DOCUMENT', 'LACK OF DOCUMENT',
       'CHEAP AND EASY ACCESS', 'CHEAP PRICE OF VEHICLE ', 'PROBLEM',
       'EXCURSION', 'MY CONDITION', 'PROBLEM OF VISA',
       'IT EASIER TO ACCESS', 'ITS EASY', 'IT SIMPLE', 'IS VERY EASY',
       'AGENT', 'FINANCIAL NEEDS', 'LESS SPENDING',
       'IT IS EASY AND CHEAP', 'EASIEST WAY', 'EASIEST WAY TO TRAVEL',
       'LESS EXPENSIVE', 'IT IS C

In [ ]:
words = ['POOR BACKGROUND', 'PRESSURE', 'HARDSHIP', 'BETTER LIFE', '',
       'SEEKING GREENER PASTURE', 'TO MAKE MONEY TO CATER FOR MY FAMILY',
       'TOO MUCH SUFFERING', 'RECESSION', 'PASTORS PROPHESY',
       'TO BETTER MY LIFE', 'FRIENDS ', 'SOCIAL MEDIA ',
       'GREENER PASTURE', 'HIGH RATE OF EURO', 'POVERTY ',
       'ECONOMIC REASONS ', 'FAMILY', 'IT IS CHEAPER',
       'LACK OF DOCUMENTS', 'LOW INCOME TO TRAVEL', 'NO TRAVEL DOCUMENT',
       'INCOMPLETE TRAVEL DOCUMENTS', 'LACK OF MONEY',
       'LACK OF TRAVEL DOCUMENTS', 'POVERTY', 'SUFFERING', 'ITS EASIER',
       'SOCIAL MEDIA', 'LACK OF TRAVEL DOCUMENT', 'LACK OF DOCUMENT',
       'CHEAP AND EASY ACCESS', 'CHEAP PRICE OF VEHICLE ', 'PROBLEM',
       'EXCURSION', 'MY CONDITION', 'PROBLEM OF VISA',
       'IT EASIER TO ACCESS', 'ITS EASY', 'IT SIMPLE', 'IS VERY EASY',
       'AGENT', 'FINANCIAL NEEDS', 'LESS SPENDING',
       'IT IS EASY AND CHEAP', 'EASIEST WAY', 'EASIEST WAY TO TRAVEL',
       'LESS EXPENSIVE', 'IT IS CHEAP', 'EASIEST, CHEAP',
       'IT IS THE EASIEST WAY', 'FRIENDS', 'IT IS VERY EASY',
       'EASY MONEY', 'EASY ROAD', 'EASY ACCESS', 'IT IS FREE ACCESS',
       'CHEAP', 'IT IS EASY', 'THE ONLY ROUTE I NO', 'PEER GROUP',
       'FRIEND', 'THE ONLY ROUTE I KNEW']

correct = ['MIS-INFORMED', 'PEER PRESSURE', 'ECONOMIC/FINANCIAL', 'ECONOMIC/FINANCIAL', np.nan,
       'ECONOMIC/FINANCIAL', 'ECONOMIC/FINANCIAL, FAMILY',
       'ECONOMIC/FINANCIAL', 'ECONOMIC/FINANCIAL', 'MIS-INFORMED',
       'ECONOMIC/FINANCIAL', 'FRIENDS', 'SOCIAL MEDIA',
       'ECONOMIC/FINANCIAL', 'ECONOMIC/FINANCIAL', 'ECONOMIC/FINANCIAL',
       'ECONOMIC/FINANCIAL', 'FAMILY', 'AFFORDABLE',
       'NO TRAVEL DOCUMENTS', 'AFFORDABLE', 'NO TRAVEL DOCUMENTS',
       'NO TRAVEL DOCUMENTS', 'AFFORDABLE',
       'NO TRAVEL DOCUMENTS', 'ECONOMIC/FINANCIAL', 'ECONOMIC/FINANCIAL', 'EASY ACCESS',
       'SOCIAL MEDIA', 'NO TRAVEL DOCUMENTS', 'NO TRAVEL DOCUMENTS',
       'AFFORDABLE, EASY ACCESS', 'AFFORDABLE', np.nan,
       'EXCURSION', 'ECONOMIC/FINANCIAL', 'NO TRAVEL DOCUMENTS',
       'EASY ACCESS', 'EASY ACCESS', 'SIMPLICITY', 'EASY ACCESS',
       'AGENTS', 'ECONOMIC/FINANCIAL', 'AFFORDABLE',
       'EASY ACCESS, AFFORDABLE', 'EASY ACCESS', 'EASY ACCESS',
       'AFFORDABLE', 'AFFORDABLE', 'EASY ACCESS, AFFORDABLE',
       'EASY ACCESS', 'FRIENDS', 'EASY ACCESS',
       'AFFORDABLE', 'EASY ACCESS', 'EASY ACCESS', 'EASY ACCESS',
       'AFFORDABLE', 'EASY ACCESS', 'THE ONLY ROUTE I KNOW', 'PEER PRESSURE',
       'FRIENDS', 'THE ONLY ROUTE I KNOW']
sample_df.Q7 = sample_df.Q7.replace(words,correct)

In [ ]:
sample_df.Q7 = sample_df.Q7.str.strip()
sample_df.Q7 = sample_df.Q7.replace('',np.nan) 
sample_df.Q7.value_counts()

ECONOMIC/FINANCIAL            19
AFFORDABLE                    13
EASY ACCESS                   13
NO TRAVEL DOCUMENTS            7
SOCIAL MEDIA                   5
PEER PRESSURE                  3
FRIENDS                        3
MIS-INFORMED                   2
EASY ACCESS, AFFORDABLE        2
THE ONLY ROUTE I KNOW          2
ECONOMIC/FINANCIAL, FAMILY     1
FAMILY                         1
AFFORDABLE, EASY ACCESS        1
EXCURSION                      1
SIMPLICITY                     1
AGENTS                         1
Name: Q7, dtype: int64

In [ ]:
Q7_df = Create(sample_df,'Q7',level=2)
Q7_df

,Edo,Delta,Kano,Katsina,Kaduna,Sokoto,Jigawa,Total
AFFORDABLE,3.0,NaN,7.0,4.0,NaN,NaN,2.0,16
AGENTS,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1
EASY ACCESS,1.0,NaN,5.0,6.0,3.0,NaN,1.0,16
ECONOMIC/FINANCIAL,7.0,9.0,NaN,NaN,3.0,NaN,1.0,20
EXCURSION,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1
FAMILY,1.0,1.0,NaN,NaN,NaN,NaN,NaN,2
FRIENDS,1.0,NaN,1.0,NaN,NaN,1.0,NaN,3
MIS-INFORMED,NaN,2.0,NaN,NaN,NaN,NaN,NaN,2
NO TRAVEL DOCUMENTS,6.0,NaN,NaN,NaN,1.0,NaN,NaN,7
PEER PRESSURE,NaN,2.0,NaN,NaN,NaN,1.0,NaN,3


###**RESPONDENTS' INTERVIEW QUESTION 8**

In [ ]:
sample_df.Q8.unique()

array(['Yes', 'No', ''], dtype=object)

In [ ]:
sample_df.Q8 = sample_df.Q8.str.strip()
sample_df.Q8 = sample_df.Q8.replace('',np.nan) 
sample_df.Q8.value_counts()

Yes    79
No      8
Name: Q8, dtype: int64

In [ ]:
Q8_df = Create(sample_df,'Q8')
Q8_df

,Edo,Delta,Kano,Katsina,Kaduna,Sokoto,Jigawa,Total
NO,NaN,3,NaN,NaN,1,NaN,4,8
YES,22.0,12,13.0,11.0,10,7.0,4,79


###**RESPONDENTS' INTERVIEW QUESTION 9**

In [ ]:
sample_df.Q9.unique()

array(['LIBYA', 'AGADEX, LIBYA, TUNISIA', 'LIBYA TO ITALY',
       'CANT REMEMBER', 'NIGER, AQLGERIA, LIBYA', 'NIGER, AGADEX, LIBYA',
       'NIGER, LIBYA', 'AGADES, LIBYA', 'AGADES, ILLELA',
       'AGADES, GATHRUM', 'AGADES, NIGER, LAKE CHAD', 'NIGER, EGYPT',
       'NIGER, AGADAS', 'NIGER, AGADES', 'NIGER', '', 'NIGER ', 'GHANA',
       'NIGER.', 'NIGER REPUBLIC/ LIBYA', 'CAMEROON,CHAD,LIBYA',
       'CAMEROON', 'NIGER, LIBYA TO ALGERIA', 'NONE',
       'NIGER, MOROCCO ROOT', 'MOROCCO/NIGER', 'ONLY NIGER',
       'CAMEROON, CHAD', 'NIGER, ALGERIA', 'CHAD, SUDAN ',
       'NIGER REPUBLIC AND CHAD', 'NIGER REPUBLIC', 'NIGER AND CHAD',
       'REPUBLIC OF NIGER', 'NIGER CHAD', 'CHAD', 'CAMAROON'],
      dtype=object)

In [ ]:
words = ['LIBYA', 'AGADEX, LIBYA, TUNISIA', 'LIBYA TO ITALY',
       'CANT REMEMBER', 'NIGER, AQLGERIA, LIBYA', 'NIGER, AGADEX, LIBYA',
       'NIGER, LIBYA', 'AGADES, LIBYA', 'AGADES, ILLELA',
       'AGADES, GATHRUM', 'AGADES, NIGER, LAKE CHAD', 'NIGER, EGYPT',
       'NIGER, AGADAS', 'NIGER, AGADES', 'NIGER', '', 'NIGER ', 'GHANA',
       'NIGER.', 'NIGER REPUBLIC/ LIBYA', 'CAMEROON,CHAD,LIBYA',
       'CAMEROON', 'NIGER, LIBYA TO ALGERIA', 'NONE',
       'NIGER, MOROCCO ROOT', 'MOROCCO/NIGER', 'ONLY NIGER',
       'CAMEROON, CHAD', 'NIGER, ALGERIA', 'CHAD, SUDAN ',
       'NIGER REPUBLIC AND CHAD', 'NIGER REPUBLIC', 'NIGER AND CHAD',
       'REPUBLIC OF NIGER', 'NIGER CHAD', 'CHAD', 'CAMAROON']

correct = ['LIBYA', 'NIGER, LIBYA, TUNISIA', 'LIBYA, ITALY',
       np.nan, 'NIGER, ALGERIA, LIBYA', 'NIGER, LIBYA',
       'NIGER, LIBYA', 'NIGER, LIBYA', 'NIGER',
       'NIGER', 'NIGER, CHAD', 'NIGER, EGYPT',
       'NIGER', 'NIGER', 'NIGER', np.nan, 'NIGER', 'GHANA',
       'NIGER', 'NIGER, LIBYA', 'CAMEROON, CHAD, LIBYA',
       'CAMEROON', 'NIGER, LIBYA, ALGERIA', np.nan,
       'NIGER, MOROCCO', 'MOROCCO,NIGER', 'NIGER',
       'CAMEROON, CHAD', 'NIGER, ALGERIA', 'CHAD, SUDAN ',
       'NIGER, CHAD', 'NIGER', 'NIGER, CHAD',
       'NIGER', 'NIGER, CHAD', 'CHAD', 'CAMEROON']
sample_df.Q9 = sample_df.Q9.replace(words,correct)

In [ ]:
sample_df.Q9 = sample_df.Q9.str.strip()
sample_df.Q9 = sample_df.Q9.replace('',np.nan) 
sample_df.Q9.value_counts()

NIGER                    50
LIBYA                     4
NIGER, LIBYA              4
NIGER, CHAD               4
CAMEROON                  4
CAMEROON, CHAD            2
NIGER, LIBYA, ALGERIA     1
CHAD, SUDAN               1
NIGER, ALGERIA            1
MOROCCO,NIGER             1
NIGER, MOROCCO            1
CAMEROON, CHAD, LIBYA     1
NIGER, LIBYA, TUNISIA     1
GHANA                     1
NIGER, EGYPT              1
NIGER, ALGERIA, LIBYA     1
LIBYA, ITALY              1
CHAD                      1
Name: Q9, dtype: int64

In [ ]:
Q9_df = Create(sample_df,'Q9',level=2)
Q9_df

,Edo,Delta,Kano,Katsina,Kaduna,Sokoto,Jigawa,Total
ALGERIA,NaN,1.0,1.0,NaN,NaN,NaN,1.0,3
CAMEROON,NaN,NaN,2.0,NaN,NaN,1.0,4.0,7
CHAD,NaN,1.0,2.0,4.0,NaN,1.0,1.0,9
EGYPT,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1
GHANA,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1
ITALY,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1
LIBYA,3.0,7.0,NaN,NaN,1.0,NaN,2.0,13
MOROCCO,NaN,NaN,NaN,NaN,2.0,NaN,NaN,2
NIGER,20.0,12.0,11.0,10.0,4.0,5.0,3.0,65
SUDAN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1


###**RESPONDENTS' INTERVIEW QUESTION 10**

In [ ]:
sample_df.Q10.unique()

array(['Truck', 'Motorbike, Bus', 'Motorbike, Foot', 'Motorbike, Truck',
       '', 'Motorbike, Foot, Boat', 'Motorbike, Car, Truck',
       'Motorbike, Boat, Bus, Truck', 'Bus, Truck', 'Motorbike',
       'Foot, Car', 'Foot, Car, Bus, Truck',
       'Motorbike, Foot, Bus, Truck', 'Foot, Car, Boat',
       'Motorbike, Bus, Truck, Animal',
       'Motorbike, Foot, Boat, Bus, Truck', 'Bus',
       'Motorbike, Foot, Truck', 'Motorbike, Foot, Car, Boat, Truck',
       'By air', 'Foot, Bus', 'Boat, Truck', 'Car', 'Boat', 'Boat, Bus',
       'Car, Bus', 'Motorbike, Foot, Boat, Truck',
       'Motorbike, Foot, Boat, Truck, Animal', 'Motorbike, Boat, Truck',
       'Motorbike, Foot, Bus', 'Motorbike, Truck, Animal',
       'Motorbike, Car'], dtype=object)

In [ ]:
sample_df.Q10 = sample_df.Q10.str.strip()
sample_df.Q10 = sample_df.Q10.replace('',np.nan) 
sample_df.Q10.value_counts()

Bus                                     15
Truck                                   14
Car                                      9
Motorbike, Foot, Truck                   8
Motorbike, Truck                         4
By air                                   3
Bus, Truck                               3
Motorbike                                3
Motorbike, Foot, Car, Boat, Truck        2
Motorbike, Bus                           2
Motorbike, Foot, Boat, Bus, Truck        2
Motorbike, Foot, Bus, Truck              2
Motorbike, Foot, Boat, Truck, Animal     2
Foot, Car                                2
Motorbike, Foot, Boat, Truck             1
Car, Bus                                 1
Motorbike, Boat, Truck                   1
Motorbike, Foot, Bus                     1
Motorbike, Truck, Animal                 1
Boat, Bus                                1
Boat                                     1
Motorbike, Foot                          1
Boat, Truck                              1
Foot, Bus  

In [ ]:
Q10_df = Create(sample_df,'Q10',level=2)
Q10_df

###**QUESTION 11**

In [ ]:
sample_df.Q11.unique()

array(['400,000', '500,000', '1,200,000', '15,000', '600,000',
       '1,000,000', 'CANT REMEMBER', 'N500,000', 'N800,000', 'N510,000',
       'N700,000', 'N600,000', '850,000', '1,250,000', '980,000',
       'I WAS TO PAY #8,500,000 IN ITALY ', '#1,100,000', '1,450,000',
       '#750,000', '#1,000,000', '', '#800,000', '#1,800,000',
       'I WAS TO PAY ON ARRIVAL AT DESTINATION ', '#850,000',
       'N150,000,000', 'N300,000', 'N1,600,000', 'N550,000', 'N1,800,000',
       'N650,000', 'ABOUT 400,000', 'ABOVE 100,000', 'ABOUT 250,000',
       '200,000', '180,000', '900,000', '350,000-500,000',
       '550,000-600,000', '1,320,000', '145', '140000', '145000',
       '120000', '150000', '60000', '80000', '170000', '65000', '160000',
       '45000', '#200,000 (3 YEARS AGO)', '#180,000', '#250,000',
       'OVER #100,000', '#285,000', '#200,000', 'OVER #200,000',
       '#160,000', 'OVER #150,000', '95,000', '7,500', '18,700', '10,000',
       '20,000', '91,000'], dtype=object)

In [ ]:
sample_df.Q11 = sample_df.Q11.str.upper()
sample_df.Q11 = sample_df.Q11.str.strip()
sample_df.Q11.value_counts()

In [ ]:
sda = sample_df.Q11
sda = pd.Series([restruct(x) for x in sda])
sda.unique()

In [ ]:
words = ['400000', '500000', '1200000', '15000', '600000', '1000000',
       'CANT REMEMBER', '800000', '510000', '700000', '850000', '1250000',
       '980000', '8500000', '1100000', '1450000', '750000', '', '1800000',
       'I WAS TO PAY ON ARRIVAL AT DESTINATION', '150000000', '300000',
       '1600000', '550000', '650000', '100000', '250000', '200000',
       '180000', '900000', '1320000', '145', '140000', '145000', '120000',
       '150000', '60000', '80000', '170000', '65000', '160000', '45000',
       '2000003', '285000', '95000', '7500', '18700', '10000', '20000',
       '91000']

correct = ['400000', '500000', '1200000', '15000', '600000', '1000000',
       '0', '800000', '510000', '700000', '850000', '1250000',
       '980000', '850000', '1100000', '1450000', '750000', np.nan, '1800000',
       '0', '1500000', '300000',
       '1600000', '550000', '650000', '100000', '250000', '200000',
       '180000', '900000', '1320000', '145', '140000', '145000', '120000',
       '150000', '60000', '80000', '170000', '65000', '160000', '45000',
       '200000', '285000', '95000', '7500', '18700', '10000', '20000',
       '91000']

sda = sda.replace(words,correct)
sample_df.Q11 = pd.to_numeric(sda)
sample_df.Q11 = sample_df.Q11/1000
sample_df.Q11.max()

In [ ]:
Q11_df = Create(sample_df,'Q11',level=1, cat=True,max=2000,range=200)
Q11_df

###**QUESTION 12**

In [ ]:
sample_df.Q12.unique()

In [ ]:
words = ['Savings', 'Friends', 'Loan/borrowed',
       'Selling of Assets or belongings', 'Family',
       'Community effort, Loan/borrowed',
       'Friends, Family, Savings, Loan/borrowed',
       'Friends, Family, Savings', 'CONNECTION MADAM',
       'Family, Savings, Selling of Assets or belongings, Loan/borrowed',
       'Friends, Family, Selling of Assets or belongings, Loan/borrowed',
       'Friends, Family, Savings, Selling of Assets or belongings, Loan/borrowed',
       'Friends, Savings, Selling of Assets or belongings, Loan/borrowed',
       'Family, Selling of Assets or belongings, Loan/borrowed', '',
       'Friends, Savings',
       'Selling of Assets or belongings, Loan/borrowed',
       'Savings, Loan/borrowed']

correct = ['Savings', 'Friends', 'Loan/borrowed',
       'Selling of Assets or belongings', 'Family',
       'Community effort, Loan/borrowed',
       'Friends, Family, Savings, Loan/borrowed',
       'Friends, Family, Savings', 'Agents',
       'Family, Savings, Selling of Assets or belongings, Loan/borrowed',
       'Friends, Family, Selling of Assets or belongings, Loan/borrowed',
       'Friends, Family, Savings, Selling of Assets or belongings, Loan/borrowed',
       'Friends, Savings, Selling of Assets or belongings, Loan/borrowed',
       'Family, Selling of Assets or belongings, Loan/borrowed', '',
       'Friends, Savings',
       'Selling of Assets or belongings, Loan/borrowed',
       'Savings, Loan/borrowed']
sample_df.Q12 = sample_df.Q12.replace(words,correct)

In [ ]:
sample_df.Q12 = sample_df.Q12.str.strip()
sample_df.Q12 = sample_df.Q12.replace('',np.nan) 
sample_df.Q12.value_counts()

In [ ]:
Q12_df = Create(sample_df,'Q12',level=2)
Q12_df

###**QUESTION 13**

In [ ]:
sample_df.Q13.unique()

In [ ]:
sample_df.Q13 = sample_df.Q13.str.strip()
sample_df.Q13 = sample_df.Q13.replace('',np.nan) 
sample_df.Q13.value_counts()

In [ ]:
Q13_df = Create(sample_df,'Q13',level=1)
Q13_df

###**QUESTION 14**

In [ ]:
sample_df.Q14.unique()

In [ ]:
sample_df.Q14 = sample_df.Q14.str.strip()
sample_df.Q14 = sample_df.Q14.replace('',np.nan) 
sample_df.Q14.value_counts()

In [ ]:
Q14_df = Create(sample_df,'Q14',level=2)
Q14_df

###**QUESTION 15**

In [ ]:
sample_df.Q15.unique()

array(['Yes', '', 'No'], dtype=object)

In [ ]:
sample_df.Q15 = sample_df.Q15.str.strip()
sample_df.Q15 = sample_df.Q15.replace('',np.nan) 
sample_df.Q15.value_counts()

In [ ]:
Q15_df = Create(sample_df,'Q15',level=1)
Q15_df

###**QUESTION 16**

In [ ]:
sample_df.Q16.unique()

In [ ]:
sample_df.Q16 = sample_df.Q16.str.strip()
sample_df.Q16 = sample_df.Q16.replace('',np.nan) 
sample_df.Q16.value_counts()

In [ ]:
Q16_df = Create(sample_df,'Q16',level=1)
Q16_df

###**QUESTION 17**

In [ ]:
sample_df.Q17.unique()

In [ ]:
sample_df.Q17 = sample_df.Q17.str.strip()
sample_df.Q17 = sample_df.Q17.replace('',np.nan) 
sample_df.Q17.value_counts()

In [ ]:
Q17_df = Create(sample_df,'Q17',level=1)
Q17_df

###**QUESTION 18**

In [ ]:
sample_df.Q18.unique()

In [ ]:
sample_df.Q18 = sample_df.Q18.str.strip()
sample_df.Q18 = sample_df.Q18.replace('',np.nan) 
sample_df.Q18.value_counts()

In [ ]:
Q18_df = Create(sample_df,'Q18',level=1)
Q18_df

###**QUESTION 19**

In [ ]:
sample_df.Q19.unique()

In [ ]:
sample_df.Q19 = sample_df.Q19.str.strip()
sample_df.Q19 = sample_df.Q19.replace('',np.nan) 
sample_df.Q19.value_counts()

In [ ]:
Q19_df = Create(sample_df,'Q19',level=1)
Q19_df

###**QUESTION 20**

In [ ]:
sample_df.Q20.unique()

array(['CUSTOM', '', 'ARMY', 'POLICE', 'NAFDAC', 'NAFDAC, IMMIGRATION',
       'NONE', 'NO', 'CIVIL DEFENCE', '_', 'POLICE/IMMIGRATION',
       'POLICE/ IMMIGRATION', 'POLICE , IMMIGRATION',
       'POLICE/ IMMIGRATION OFFICERS', 'IMMIGRATION', 'YES',
       'IMMIGRATION OFFICER', 'NIGER OFFICIALS', 'IMMIGRATION OFFICERS',
       'IMMIGRATION AND CUSTOM', 'LIBIYA', 'ALGERIA', 'NIGER', 'CHAD'],
      dtype=object)

In [ ]:
words = ['CUSTOM', '', 'ARMY', 'POLICE', 'NAFDAC', 'NAFDAC, IMMIGRATION',
       'NONE', 'NO', 'CIVIL DEFENCE', '_', 'POLICE/IMMIGRATION',
       'POLICE/ IMMIGRATION', 'POLICE , IMMIGRATION',
       'POLICE/ IMMIGRATION OFFICERS', 'IMMIGRATION', 'YES',
       'IMMIGRATION OFFICER', 'NIGER OFFICIALS', 'IMMIGRATION OFFICERS',
       'IMMIGRATION AND CUSTOM', 'LIBIYA', 'ALGERIA', 'NIGER', 'CHAD']
correct = ['CUSTOM', '', 'ARMY', 'POLICE', 'NAFDAC', 'NAFDAC, IMMIGRATION',
       'NONE', 'NONE', 'CIVIL DEFENCE', np.nan, 'POLICE, IMMIGRATION',
       'POLICE, IMMIGRATION', 'POLICE, IMMIGRATION',
       'POLICE, IMMIGRATION', 'IMMIGRATION', 'YES',
       'IMMIGRATION', 'NIGER OFFICIALS', 'IMMIGRATION',
       'IMMIGRATION, CUSTOM', 'LIBIYA OFFICIALS', 'ALGERIA OFFICIALS', 'NIGER OFFICIALS', 'CHAD OFFICIALS']
sample_df.Q20 = sample_df.Q20.replace(words,correct)

In [ ]:
sample_df.Q20 = sample_df.Q20.str.strip()
sample_df.Q20 = sample_df.Q20.replace('',np.nan) 
sample_df.Q20.value_counts()

In [ ]:
Q20_df = Create(sample_df,'Q20',level=2)
Q20_df

###**QUESTION 21**

In [ ]:
sample_df.Q21.unique()

In [ ]:
sample_df.Q21 = sample_df.Q21.str.strip()
sample_df.Q21 = sample_df.Q21.replace('',np.nan) 
sample_df.Q21.value_counts()

In [ ]:
Q21_df = Create(sample_df,'Q21',level=1)
Q21_df

###**QUESTION 22**

In [ ]:
sample_df.Q22.unique()

In [ ]:
sample_df.Q22 = sample_df.Q22.str.strip()
sample_df.Q22 = sample_df.Q22.replace('',np.nan) 
sample_df.Q22.value_counts()

In [ ]:
Q22_df = Create(sample_df,'Q22',level=1)
Q22_df

###**QUESTION 23**

In [ ]:
sample_df.Q23.unique()

In [ ]:
sample_df.Q23 = sample_df.Q23.str.strip()
sample_df.Q23 = [x.replace('#','N') for x in sample_df.Q23]
sample_df.Q23 = sample_df.Q23.replace('',np.nan) 
sample_df.Q23.value_counts()

In [ ]:
Q23_df = Create(sample_df,'Q23',level=1)
Q23_df

In [ ]:
 Q23_df = Q23_df.reset_index()
 Q23_df.to_csv('/content/drive/MyDrive/Colab Notebooks/KII Analysis/{}_Summary.csv'.format('Q23'))
 Q23_df

,index,Edo,Delta,Kano,Katsina,Kaduna,Sokoto,Jigawa,Total
0,"1,000",NaN,NaN,NaN,NaN,NaN,1.0,NaN,1
1,ASSIST US TO ESCAPE FROM BOKO HARAM,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1
2,ASSIST US TO ESCAPE FROM BOKO HARAM TERRORIST,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1
3,ASSIST US TO ESCAPE FROM BOKO HARAM TERRORIST ...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1
4,BY CHECKING ALL THE DOCUMENTS,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1
5,COLLECTED MONEY,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1
6,"EXIST MONEY N10,000",NaN,NaN,NaN,1.0,NaN,NaN,NaN,1
7,EXTORTION OF OUR MONEY,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1
8,GUIDENCE AND COUNCILLING,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1
9,I PAID MONEY,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1


###**QUESTION 24**


In [ ]:
sample_df.Q24.unique()

In [ ]:
words = ['Threat to life', 'KIDNAPPING', '', 'TOO MUCH SUFFERING',
       'SHADES IN DESERTS, STARVATION', 'STARVATION,INTIMIDATIOM',
       'STARVATION', 'NO WATER NO FOOD', 'KIDNAPPING,STARVATION',
       'KIDNAPPING, BEATING', 'HUNGER', 'BRUTALITY',
       'ALMOST ALL MY FRIENDS DIED', 'LACK OF FREEDOM , EXTORTION',
       'LACK OF WATER', 'LACK OF WATER , LACK OF MEDICATION',
       'LACK OF FREEDOM, LACK OF WATER ',
       'INABILITY TO RETURN BACK ON MY OWN', 'DRINKING OF URINE',
       'HUNGER, LACK OF WATER', 'EXTORTION',
       'STARVATION, HARRASSMENT, EXTORTION',
       'MANIPULATION, EXTORTION, OPPRESSION', 'HUNGER, FATIQUE',
       'Sexual abuse', 'LACK OF FREEDOM',
       'LOST ALL I HAVE ON THE JOURNY EVEN MY PHONE WAS STOLEN',
       'LACK OF FUND', 'HUNGER, LACK OF WATER AND ARMED ROBBERS',
       'Forced labour', 'DEPORTATION', 'HUNGER, HUMAN TORTURE', 'NO JOB',
       'NO FREEDOM', 'AGENT', 'LACK OF FOOD',
       'DIFFICULTY OF PASSING SAHARA DESERT', 'NONE',
       'MALTREATING BY ALGERIAN OFFICIALS',
       'MALTREATMENT BY LIBYAN OFFICIALS', 'INADEQUATE CAPITAL',
       'Loss of property', 'LONG ROUTE TO ALGERIA', 'SERIOUS HUNGER',
       'LONG ROUTE, THIRST AND HUNGER', 'SOME PEOPLE DIED ON TRANSIT',
       'Imprisonment']
correction = ['Threat to life', 'KIDNAPPING', '', 'SUFFERING',
       'STARVATION', 'STARVATION, INTIMIDATION',
       'STARVATION', 'STARVATION', 'KIDNAPPING, STARVATION',
       'KIDNAPPING, BEATING', 'STARVATION', 'BRUTALITY',
       'DEATH', 'LACK OF FREEDOM , EXTORTION',
       'LACK OF WATER', 'LACK OF WATER , LACK OF MEDICATION',
       'LACK OF FREEDOM, LACK OF WATER ',
       'LACK OF FREEDOM', 'LACK OF WATER',
       'STARVATION, LACK OF WATER', 'EXTORTION',
       'STARVATION, HARRASSMENT, EXTORTION',
       'MANIPULATION, EXTORTION, OPPRESSION', 'STARVATION, FATIQUE',
       'Sexual abuse', 'LACK OF FREEDOM',
       'LOSS OF PROPERTY',
       'LACK OF FUND', 'STARVATION, LACK OF WATER, LOSS OF PROPERTY',
       'Forced labour', 'DEPORTATION', 'STARVATION, HUMAN TORTURE', 'NO JOB',
       'LACK OF FREEDOM', 'AGENTS', 'STARVATION',
       'DIFFICULTY OF PASSING SAHARA DESERT', 'NONE',
       'MALTREATING BY ALGERIAN OFFICIALS',
       'MALTREATMENT BY LIBYAN OFFICIALS', 'INADEQUATE CAPITAL',
       'Loss of property', 'LONG ROUTE TO ALGERIA', 'STARVATION',
       'LONG ROUTE, STARVATION', 'DEATH',
       'Imprisonment']
sample_df.Q24 = sample_df.Q24.replace(words, correction)

In [ ]:
sample_df.Q24 = sample_df.Q24.str.strip() 
sample_df.Q24 = sample_df.Q24.replace('',np.nan) 
sample_df.Q24.value_counts()

In [ ]:
Q24_df = Create(sample_df,'Q24',level=2)
Q24_df

In [ ]:
 Q24_df = Q24_df.reset_index()
 Q24_df.to_csv('/content/drive/MyDrive/Colab Notebooks/KII Analysis/{}_Summary.csv'.format('Q24'))
 Q24_df

###**QUESTION 25**

In [ ]:
sample_df.Q25.unique()

In [ ]:
words = ['DEPORTED', 'SUFFERING TOO MUCH', 'RISK OF CROSSING THE SEA',
       'NIGERIA IS THE BEST COUNTRY FOR ME', '',
       'I WAS DROPPED, BEBORE I GOT TO MY DESTINATION', 'DEPORT',
       'STRESSFUL CONDITION', 'IT WAS TOUGH', 'ZONOPHOBIC ATTACK',
       'I SPENT 2 YEARS SUFFERING',
       'THE PLACE IS VERY RISKY AND DANGEROUS FOR A WOMAN TO STAY',
       '2020', 'I WAS CAPTURED AND RETURNED', 'FRUSTRATION', 'FRUSTRATED',
       'DEPORTATION', 'KIDNAPPING', 'CAPTURED ON THE SEA',
       'CAUGHT IN THE SEA', 'I WAS CAPTURED', 'TO SEE MY FAMILY',
       'TO COME BACK TO MY FAMILY', 'NO JOB', 'NO WORK',
       'SUFFERED MUCH THERE', 'MY CONTRACT EXPIRED', 'SAME WAY',
       'HUNGER AND SICKNESS', 'LACK OF WORK', 'BECAUSE OF HARDSHIP',
       'THE CONTRACT EXPIRED', 'I WAS RETURNED BY NIGER POLICE',
       'RETURNED BY LIBYA OFFICERS', 'FORCED', 'FORCED RETURN', 'FORCED ',
       'I WANT TO LIVE WITH MY FAMILY', 'TWO OF MY FRIENDS WERE KILLED',
       'TIRED OF STAYING AS I"M GETTING OLD',
       'THE WAR IN LIBYA MADE ME RETURN', 'EXPLOITATION BY AGENTS',
       'THREAT TO LIFE', 'TO PICK MY FEMALE CHILDREN',
       'INSECURITY IN THE COUNTRY', 'THE PLACE IS NO LONGER SECURED',
       'I WAS TIRED OF ALGERIA, NO REST MIND', 'VISIT FRIENDS',
       'TO VISIT MY FRIEND ', 'TO VISIT MY FAMILY', 'VISIT TO FAMILY']


correction = ['DEPORTED', 'SUFFERING TOO MUCH', 'RISK OF CROSSING THE SEA',
       'NIGERIA IS THE BEST COUNTRY FOR ME', '',
       'I WAS DROPPED, BEFORE I GOT TO MY DESTINATION', 'DEPORTED',
       'STRESSFUL CONDITION', 'IT WAS TOUGH', 'ZONOPHOBIC ATTACKS',
       'I SPENT 2 YEARS SUFFERING',
       'THE PLACE IS VERY RISKY AND DANGEROUS FOR A WOMAN TO STAY',
       np.nan, 'I WAS CAPTURED AND RETURNED', 'FRUSTRATED', 'FRUSTRATED',
       'DEPORTATED', 'KIDNAPPED', 'CAPTURED ON THE SEA',
       'CAPTURED ON THE SEA', 'I WAS CAPTURED', 'RETURN TO MY FAMILY',
       'RETURN TO MY FAMILY', 'NO JOB', 'NO WORK',
       'SUFFERED MUCH THERE', 'MY CONTRACT EXPIRED', 'SAME WAY',
       'HUNGER AND SICKNESS', 'LACK OF WORK', 'BECAUSE OF HARDSHIP',
       'CONTRACT EXPIRED', 'I WAS RETURNED BY NIGER POLICE',
       'RETURNED BY LIBYA OFFICERS', 'FORCED TO RETURN', 'FORCED TO RETURN', 'FORCED TO RETURN',
       'RETURN TO MY FAMILY', 'TWO OF MY FRIENDS WERE KILLED',
       "TIRED OF STAYING AS I'M GETTING OLD",
       'THE WAR IN LIBYA MADE ME RETURN', 'EXPLOITATION BY AGENTS',
       'THREAT TO LIFE', 'TO PICK MY FEMALE CHILDREN',
       'INSECURITY IN THE COUNTRY', 'THE PLACE IS NO LONGER SECURED',
       'I WAS TIRED OF ALGERIA, NO REST OF MIND', 'TO VISIT MY FRIENDS',
       'TO VISIT MY FRIENDS', 'TO VISIT MY FAMILY', 'TO VISIT MY FAMILY']
       
sample_df.Q25 = sample_df.Q25.replace(words, correction)

In [ ]:
sample_df.Q25 = sample_df.Q25.str.strip() 
sample_df.Q25 = sample_df.Q25.replace('',np.nan) 
sample_df.Q25.value_counts()

In [ ]:
Q25_df = Create(sample_df,'Q25',level=1)
Q25_df

In [ ]:
 Q25_df = Q25_df.reset_index()
 Q25_df.to_csv('/content/drive/MyDrive/Colab Notebooks/KII Analysis/{}_Summary.csv'.format('Q25'))
 Q25_df

###**QUESTION 26**

In [ ]:
sample_df.Q26.unique()

In [ ]:
words = ['FLIGHT (IOM)', 'IOM', '', 'BY ROAD THROUGH SOKOTO', 'UN POLICE',
       'BY AIR', 'NIGERIA GOVERNMENT ', 'FEDERAL GOVERNMENT',
       'I WAS DEPORTED TO GHANA AND CAME BACK THROUGH COTONOU',
       'PROPHET T.B JOSHUA CHARTERED FLIGHT ',
       'THROUGH MURITALA MOHAMMED AIRPORT',
       'THROUGH NIGERIA CHARTED FLIGHT', 'ON MY OWN',
       'IOM CHARTERED FLIGHT', 'HARDSHIP', 'THROUGH NIGERIAN GOVERNMENT',
       'THROUGH NIGERIAN CHARTERED FLIGHT', 'THROUGH IOM', 'BY BUS',
       'BUS', 'BY ROAD', 'BY AIR TO KANO', 'CAR THROUH BORDER',
       'THROUGH FLIGHT', 'THROUGHT IOM', 'IOM OFFICIALS', 'IOM FLIGHT',
       'I CAME BACK THROUGH PUBLIC TRANSPORT WITH BRIBE',
       'THROUGH AGADEZ DESERT', 'THE SAME PROCESS',
       'I FOLLOWED SEVERAL COMMUNITIES TO GET TO NIGER',
       'I FOLLOWED ONE WOMAN FROM NIGER REPUBLIC', 'I CAME BY ROAD',
       'THROUGH AGENTS AND LOCAL COMMUNITIES',
       'I SPENT SEVERAL WEEKS WITH HELP FROM VILLAGERS',
       'THROUGH THE NORTHWEST BORDER', 'TRHOUGH', 'BY RAIL',
       'NORTHWEST ROUTE', 'BY THE ROUTE', 'THROUGH BUS']

correct = ['IOM SPONSORED FLIGHT', 'IOM SPONSORED FLIGHT', '', 'BY ROAD THROUGH SOKOTO', 'UN POLICE',
       'BY AIR', 'NIGERIA GOVERNMENT ', 'FEDERAL GOVERNMENT',
       'I WAS DEPORTED TO GHANA AND CAME BACK THROUGH COTONOU',
       'PROPHET T.B JOSHUA CHARTERED FLIGHT ',
       'THROUGH MURITALA MOHAMMED AIRPORT',
       'THROUGH NIGERIA CHARTED FLIGHT', 'ON MY OWN',
       'IOM CHARTERED FLIGHT', 'HARDSHIP', 'THROUGH NIGERIAN GOVERNMENT',
       'THROUGH NIGERIAN CHARTERED FLIGHT', 'IOM SPONSORED FLIGHTS', 'BY BUS',
       'BUS', 'BY ROAD', 'BY AIR TO KANO', 'CAR THROUH BORDER',
       'THROUGH FLIGHT', 'IOM SPONSORED FLIGHTS', 'IOM SPONSORED FLIGHTS', 'IOM SPONSORED FLIGHTS',
       'I CAME BACK THROUGH PUBLIC TRANSPORT WITH BRIBE',
       'THROUGH AGADEZ DESERT', 'THE SAME PROCESS',
       'I FOLLOWED SEVERAL COMMUNITIES TO GET TO NIGER',
       'I FOLLOWED ONE WOMAN FROM NIGER REPUBLIC', 'I CAME BY ROAD',
       'THROUGH AGENTS AND LOCAL COMMUNITIES',
       'I SPENT SEVERAL WEEKS WITH HELP FROM VILLAGERS',
       'THROUGH THE NORTHWEST BORDER', 'TRHOUGH', 'BY RAIL',
       'NORTHWEST ROUTE', 'BY THE ROUTE', 'THROUGH BUS']

sample_df.Q26 = sample_df.Q26.replace(words,correct)

In [ ]:
sample_df.Q26 = sample_df.Q26.str.strip()
sample_df.Q26 = sample_df.Q26.replace('',np.nan) 
sample_df.Q26.value_counts()

In [ ]:
Q26_df = Create(sample_df,'Q26',level=1)
Q26_df

In [ ]:
 Q26_df = Q26_df.reset_index()
 Q26_df.to_csv('/content/drive/MyDrive/Colab Notebooks/KII Analysis/{}_Summary.csv'.format('Q26'))
 Q26_df

,index,Edo,Delta,Kano,Katsina,Kaduna,Sokoto,Jigawa,Total
0,BUS,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1
1,BY AIR,NaN,1.0,NaN,1.0,3.0,NaN,NaN,5
2,BY AIR TO KANO,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1
3,BY BUS,NaN,NaN,NaN,NaN,3.0,NaN,NaN,3
4,BY RAIL,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1
5,BY ROAD,NaN,NaN,NaN,1.0,1.0,NaN,NaN,2
6,BY ROAD THROUGH SOKOTO,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1
7,BY THE ROUTE,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1
8,CAR THROUH BORDER,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1
9,FEDERAL GOVERNMENT,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1


###**QUESTION 27**

In [ ]:
sample_df.Q27.unique()

In [ ]:
sample_df.Q27 = sample_df.Q27.str.strip()
sample_df.Q27 = sample_df.Q27.replace('',np.nan) 
sample_df.Q27.value_counts()

In [ ]:
Q27_df = Create(sample_df,'Q27',level=1)
Q27_df

###**QUESTION 28**

In [ ]:
sample_df.Q28.unique()

In [ ]:
wrong = ['FIX OUR ECONOMY', 'MIGRANTS SHOULD TELL THEIR STORY',
       'SENSITIZATION', '',
       'SET UP SKILL ACQUISITION CENTRE TO CARRY OUT AWARENESS',
       'TO GIVE YOUTHS SOMETHING TANGIBLE TO DO', 'ASSISTING PEOPLE',
       'EMPOWER PEOPLE WITH BUSINESS', 'ESTABLISG BUSINESS FOR PEOPLE',
       'TELLING STORIES AND SENSITIZATION', 'EMPOWERMENT ',
       'QUICK TRAVELLING DOCUMENT', 'ACCESS TO QUICK TRAVEL DOCUMENT',
       'PROVISION OF EMPLOYMENT ', 'EASY TRAVEL DOCUMENT',
       'ENGAGE THE YOUTH TO KEEP THEM BUSY ', 'AWARENESS',
       'AWARENESS RAISING ', 'INFORMATION',
       'GOVERNMENT SHOULD REDUCE THE STRESS OF GETTING VISA',
       'MORE JOB OPPORTUNITIES, YOUTH EMPOWERMENT ', 'CREATE MORE JOBS',
       'REDUCE THE PROBLEM AROUND GETTING VISA',
       'CREATE JOB OPPORTUNITIES ', 'EMPOWERMENT', 'EMPOWERMENT OF YOUTH',
       'EMPOERMENT OF YOUTHS', 'CREATE AWARENESS',
       'FININACIAL ASSISTANCE TO THE POOR ', 'ASIST US WITH JOB',
       'CREATE AWARENESS, JOB AND FINANCIAL ASSISTANCE ',
       'CREATE JOB OPPORTUNITY',
       'EMPOWER YOUTH AND WOMEN WHO ARE JOBLESS',
       'MAKE FOOD AVAILABLE TO CITIZENS',
       'INCREASE THE STANDARD OF LIVING AND REDUCE INFLATION',
       'CREATE JOB OPPORTUNITIES',
       'TO HELP SECURITY AGENCIES AT THE BORDER',
       'TO CREATE JOB OPPORTUNITY', 'REDUCE COST OF TRAVELLING',
       'CREATE JOB', 'POVERTY ALLEVIATION',
       'SENSITIZATION ON THE DANGERS AND DIFFICULTIES',
       'TO EDUCATE PEOPLE', 'PROVIDING JOBS', 'TO STOP AGENTS',
       'PROVIDE JOB OPPORTUNITY', 'SHOULD BE RECOMMENDED NOT TO TRY',
       'IT IS RECOMMENDED TO STAY IN NIGERIA', 'NOT TO ENCOURAGE',
       'STAYING IN NIGERIA IS THE BEST', 'STAYING IN NIGERIA IS BETTER',
       'IT IS VERY RISKY', 'IT IS DANGEROUS TO LIFE AND PROPERTIES',
       'IT IS VERY RISKY TO LIFE', 'IT IS VERY DANGEROUS',
       'ENCOURAGE BORDER OFFICIALS TO WORK HARD', 'IT IS NOT GOOD',
       'GOOD GOVERNANCE', 'GOVERNMENT SHOULD PROVIDE JOB',
       'GOVERNMENT SHOULD MAKE TRAVEL EASY',
       'GOVERNMENT SHOULD MAKE THINGS EASY',
       'MAKE TRAVEL DOCUMENTS CHEAP', 'BETTER GOVERNMENT',
       'EDUCATION AND EMPOLYMENT', 'GOOD WORK', 'GOOD JOBS/EMPLOYMENT',
       'TO FOLLOW REGULAR WAY ',
       'TO STOP PEOPLE FROM FOLLOWING IRREGULAR ROUTE',
       'ORIENTATION OF PEOPLE']

correct =     ['FIX OUR ECONOMY', 'MIGRANTS SHOULD TELL THEIR STORY',
       'SENSITIZATION', '',
       'SET UP SKILL ACQUISITION CENTRE TO CARRY OUT AWARENESS',
       'TO GIVE YOUTHS SOMETHING TANGIBLE TO DO', 'ASSISTING PEOPLE',
       'EMPOWER PEOPLE WITH BUSINESS', 'ESTABLISG BUSINESS FOR PEOPLE',
       'TELLING STORIES AND SENSITIZATION', 'EMPOWERMENT ',
       'QUICK TRAVELLING DOCUMENT', 'ACCESS TO QUICK TRAVEL DOCUMENT',
       'PROVISION OF EMPLOYMENT ', 'EASY TRAVEL DOCUMENT',
       'ENGAGE THE YOUTH TO KEEP THEM BUSY ', 'AWARENESS',
       'AWARENESS RAISING ', 'INFORMATION',
       'GOVERNMENT SHOULD REDUCE THE STRESS OF GETTING VISA',
       'MORE JOB OPPORTUNITIES, YOUTH EMPOWERMENT ', 'CREATE MORE JOBS',
       'REDUCE THE PROBLEM AROUND GETTING VISA',
       'CREATE JOB OPPORTUNITIES ', 'EMPOWERMENT', 'EMPOWERMENT OF YOUTH',
       'EMPOERMENT OF YOUTHS', 'CREATE AWARENESS',
       'FININACIAL ASSISTANCE TO THE POOR ', 'ASIST US WITH JOB',
       'CREATE AWARENESS, JOB AND FINANCIAL ASSISTANCE ',
       'CREATE JOB OPPORTUNITY',
       'EMPOWER YOUTH AND WOMEN WHO ARE JOBLESS',
       'MAKE FOOD AVAILABLE TO CITIZENS',
       'INCREASE THE STANDARD OF LIVING AND REDUCE INFLATION',
       'CREATE JOB OPPORTUNITIES',
       'HELP SECURITY AGENCIES AT THE BORDER',
       'CREATE JOB OPPORTUNITY', 'REDUCE COST OF TRAVELLING',
       'CREATE JOB OPPORTUNITY', 'POVERTY ALLEVIATION',
       'SENSITIZATION ON THE DANGERS AND DIFFICULTIES',
       'TO EDUCATE PEOPLE', 'CREATE JOB OPPORTUNITY', 'TO STOP AGENTS',
       'CREATE JOB OPPORTUNITY', 'SHOULD BE RECOMMENDED NOT TO TRY',
       'IT IS RECOMMENDED TO STAY IN NIGERIA', 'NOT TO ENCOURAGE',
       'STAYING IN NIGERIA IS THE BEST', 'STAYING IN NIGERIA IS BETTER',
       'IT IS VERY RISKY', 'IT IS DANGEROUS TO LIFE AND PROPERTIES',
       'IT IS VERY RISKY TO LIFE', 'IT IS VERY DANGEROUS',
       'ENCOURAGE BORDER OFFICIALS TO WORK HARD', 'IT IS NOT GOOD',
       'GOOD GOVERNANCE', 'GOVERNMENT SHOULD PROVIDE JOBS',
       'GOVERNMENT SHOULD MAKE TRAVEL EASY',
       'GOVERNMENT SHOULD MAKE THINGS EASY',
       'MAKE TRAVEL DOCUMENTS CHEAP', 'BETTER GOVERNMENT',
       'EDUCATION AND EMPOLYMENT', 'GOOD WORK', 'GOOD JOBS/EMPLOYMENT',
       'TO FOLLOW REGULAR WAY ',
       'TO STOP PEOPLE FROM FOLLOWING IRREGULAR ROUTE',
       'ORIENTATION OF PEOPLE']
sample_df.Q28 = sample_df.Q28.replace(wrong,correct)

In [ ]:
sample_df.Q28 = sample_df.Q28.str.strip()
sample_df.Q28 = sample_df.Q28.replace('',np.nan) 
sample_df.Q28.value_counts()

In [ ]:
Q28_df = Create(sample_df,'Q28',level=1)
Q28_df

In [ ]:
 Q28_df = Q28_df.reset_index()
 Q28_df.to_csv('/content/drive/MyDrive/Colab Notebooks/KII Analysis/{}_Summary.csv'.format('Q28'))
 Q28_df

###**QUESTION 29**

In [ ]:
sample_df.Q29.unique()

In [ ]:
sample_df.Q29 = sample_df.Q29.str.strip()
sample_df.Q29 = sample_df.Q29.replace('',np.nan) 
sample_df.Q29.value_counts()

In [ ]:
Q29_df = Create(sample_df,'Q29',level=1)
Q29_df

###**QUESTION 30**

In [ ]:
sample_df.Q30.unique()

array(['Yes', '', 'No'], dtype=object)

In [ ]:
sample_df.Q30 = sample_df.Q30.str.strip()
sample_df.Q30 = sample_df.Q30.replace('',np.nan) 
sample_df.Q30.value_counts()

In [ ]:
Q30_df = Create(sample_df,'Q30',level=1)
Q30_df

###**QUESTION 31**

In [ ]:
sample_df.Q31.unique()

In [ ]:
sample_df.Q31 = sample_df.Q31.str.strip()
sample_df.Q31 = sample_df.Q31.replace('',np.nan) 
sample_df.Q31.value_counts()

In [ ]:
Q31_df = Create(sample_df,'Q31',level=1)
Q31_df